In [0]:
# =============================================================
# STREAMING INPE FOCOS (10min) — Databricks + Delta Live
# =============================================================
# Autor: Rubens F.
# Atualização: 2025-11-12
# Melhoria:
# - Baixa apenas os arquivos referentes ao dia informado (AAAAMMDD)
# =============================================================

import os, re, time, requests, threading
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name

# ==============================
# CONFIGURAÇÕES DE DATA E URL
# ==============================
DATA_ALVO = "20251112"  # <-- altere aqui o dia de interesse (AAAAMMDD)
BASE_URL = "https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/10min/"
HEADERS = {"User-Agent": "inpe-focos-stream/1.3"}
INTERVALO = 600  # segundos entre checagens (10 min)
MIN_TAMANHO = 1024  # bytes mínimos para considerar CSV válido

# ==============================
# PATHS DO VOLUME DATABRICKS
# ==============================
VOLUME_BASE = "/Volumes/datamasters/raw/raw_inpe/stream"
LANDING_PATH = f"{VOLUME_BASE}/inpe_in/{DATA_ALVO}"  # separa por dia
CHECKPOINT_PATH = f"{VOLUME_BASE}/_checkpoints/inpe_focos/{DATA_ALVO}"
SCHEMA_PATH = f"{VOLUME_BASE}/_schemas/inpe_focos/{DATA_ALVO}"
DESTINO_DELTA = f"/Volumes/datamasters/raw/raw_inpe/{DATA_ALVO}"

# Cria pastas se não existirem
for path in [LANDING_PATH, CHECKPOINT_PATH, SCHEMA_PATH]:
    dbutils.fs.mkdirs(f"dbfs:{path}")

# ==============================
# SPARK SESSION
# ==============================
spark = SparkSession.builder.appName(f"INPE-Focos-Stream-{DATA_ALVO}").getOrCreate()

# ==============================
# FUNÇÃO: LISTAR CSVs DO SITE
# ==============================
def listar_csvs_remotos():
    r = requests.get(BASE_URL, headers=HEADERS, timeout=30)
    r.raise_for_status()
    arquivos = re.findall(r'"([^"]+\.csv)"', r.text)
    # Filtra somente os que contenham a data desejada
    return [a for a in arquivos if DATA_ALVO in a]

# ==============================
# FUNÇÃO: BAIXAR NOVOS ARQUIVOS
# ==============================
def baixar_csvs_continuamente():
    baixados = set()
    print(f"Iniciando captura do INPE ({DATA_ALVO}) a cada {INTERVALO}s...")

    while True:
        try:
            for nome in listar_csvs_remotos():
                if nome not in baixados:
                    url = BASE_URL + nome
                    r = requests.get(url, headers=HEADERS, timeout=60)
                    if r.status_code == 200 and len(r.content) > MIN_TAMANHO:
                        destino = f"dbfs:{LANDING_PATH}/{nome}"
                        tmpfile = f"/tmp/{nome}"
                        with open(tmpfile, "wb") as f:
                            f.write(r.content)
                        dbutils.fs.cp(f"file:{tmpfile}", destino)
                        os.remove(tmpfile)
                        print(f"[✔] Baixado {nome}")
                        baixados.add(nome)
                    else:
                        print(f"[✖] Falha {nome}: tamanho_invalido")
            time.sleep(INTERVALO)
        except Exception as e:
            print(f"[ERRO] {e}")
            time.sleep(120)

# ==============================
# FUNÇÃO: INICIAR STREAM
# ==============================
def iniciar_stream():
    print(f"Iniciando stream: {LANDING_PATH} → {DESTINO_DELTA}")
    df_stream = (
        spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("cloudFiles.schemaLocation", SCHEMA_PATH)
        .option("header", True)
        .load(LANDING_PATH)
        .withColumn("arquivo_origem", input_file_name())
    )

    (
        df_stream.writeStream
        .format("delta")
        .option("checkpointLocation", CHECKPOINT_PATH)
        .outputMode("append")
        .start(DESTINO_DELTA)
    )

# ==============================
# EXECUÇÃO PRINCIPAL
# ==============================
if __name__ == "__main__":
    t = threading.Thread(target=baixar_csvs_continuamente, daemon=True)
    t.start()
    iniciar_stream()
    while True:
        time.sleep(300)
